# Biopython Annotated Chromosomes Diagram

In [1]:
import pandas as pd
from collections import OrderedDict
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html, Input, Output, State, dash_table, callback_context

import csv

from flask import Flask
import sqlite3
import numpy as np
import plotly.graph_objs as go

import os
import zlib
import subprocess
import zlib
import uuid

from collections import OrderedDict

import dash
from dash import no_update # https://community.plotly.com/t/error-expected-the-output-type-to-be-a-list-or-tuple-but-got-none/34744/6
from flask import Flask, send_from_directory
from urllib.parse import quote as urlquote

from io import StringIO
from Bio import SeqIO
from Bio import Phylo
from Bio import AlignIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

# import numpy as np
from io import BytesIO
import base64


# import pandas as pd
from plotly.tools import mpl_to_plotly
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [2]:
import gzip
import os

import pandas as pd
from Bio import SeqIO

# Use this function to generate the chromosome.tsv file

In [ ]:
chrom_list = []
for file_name in os.listdir('./'):
    suffix = file_name.split(".")[-1]
    if file_name[-4:] != "a.gz":
        continue
    if "Athaliana" in file_name:
        genotype_id = "Arabidopsis"
    else:
        genotype_id = "B73v4"

    fasta_file = gzip.open(file_name, mode='rt')
    for chrom in SeqIO.parse(fasta_file, "fasta"):
        # Check if the sequence ID could be coerced into an integer, which is likely to be a chromosome
        if not any([chrom.id.replace(prefix, "").isdigit() for prefix in ["Chr", "chr", "C", "c"]]):
            continue
        # Add species and genotype columns for a better annotation
        chrom_list.append([file_name, genotype_id, chrom.id, len(chrom.seq)])

# Dataframe that contains each chromosome lenght
df = pd.DataFrame(chrom_list)
df.columns = ["genome", "genotype_id","chr", "len"]
df.to_csv("chromosome.tsv", index=False, sep="\t")

In [40]:
df = pd.read_csv("chromosome.tsv", sep="\t")
df

,genome,genotype_id,chr,len
0,Athaliana_167_TAIR10.fa.gz,Arabidopsis,Chr1,30427671
1,Athaliana_167_TAIR10.fa.gz,Arabidopsis,Chr2,19698289
2,Athaliana_167_TAIR10.fa.gz,Arabidopsis,Chr3,23459830
3,Athaliana_167_TAIR10.fa.gz,Arabidopsis,Chr4,18585056
4,Athaliana_167_TAIR10.fa.gz,Arabidopsis,Chr5,26975502
5,Zmays_493_APGv4.fa.gz,B73v4,1,307041717
6,Zmays_493_APGv4.fa.gz,B73v4,2,244442276
7,Zmays_493_APGv4.fa.gz,B73v4,3,235667834
8,Zmays_493_APGv4.fa.gz,B73v4,4,246994605
9,Zmays_493_APGv4.fa.gz,B73v4,5,223902240


In [19]:
gene_list = ["Zm00001d020673",
    "Zm00001d020562",
    "Zm00001d020177",
    "Zm00001d024875",
    "Zm00001d006622",
    "Zm00001d025200",
    "Zm00001d002610",
    "Zm00001d024515",
    "Zm00001d007918",
    "Zm00001d034095",
    "Zm00001d007112"]

In [31]:
con = sqlite3.connect("../SQNce.db")

In [32]:
def get_gene_coordinates(genotype, gene_id):
    con = sqlite3.connect("../SQNce.db") # deploy with this
    cursorObj = con.cursor()
    df = pd.read_sql_query('''SELECT * 
                    FROM gene_coordinates 
                    WHERE genotype_id = "{0}"
                    AND gene_id = "{1}"
                    '''.format(genotype, gene_id), con)
    return(df)

In [39]:
genotype = "B73v4"
coords = pd.DataFrame()
for gene in gene_list:
    coords = pd.concat([df, get_gene_coordinates(genotype, gene)])
coords

,gene_id,genotype_id,gene_chr,gene_start,gene_end,gene_orientation
0,Zm00001d020673,B73v4,7,127586707,127588589,-
0,Zm00001d020562,B73v4,7,122351840,122352624,-
0,Zm00001d020177,B73v4,7,97680746,97682402,+
0,Zm00001d024875,B73v4,10,91796481,91798115,+
0,Zm00001d006622,B73v4,2,212780800,212783824,+
0,Zm00001d025200,B73v4,10,108836325,108838302,+
0,Zm00001d002610,B73v4,2,16708568,16710668,+
0,Zm00001d024515,B73v4,10,75531512,75533071,-
0,Zm00001d007918,B73v4,2,242408695,242410359,+
0,Zm00001d034095,B73v4,1,283893649,283895468,+


In [48]:
features = []
for row in coords[coords["gene_chr"]==str(2)].values:
    features.append((row[3], row[3], None, row[0], "black",))
features

[(212780800, 212780800, None, 'Zm00001d006622', 'black'),
 (16708568, 16708568, None, 'Zm00001d002610', 'black'),
 (242408695, 242408695, None, 'Zm00001d007918', 'black'),
 (222488036, 222488036, None, 'Zm00001d007112', 'black'),
 (222488036, 222488036, None, 'Zm00001d007112', 'black')]

In [75]:
from reportlab.lib.units import cm
from Bio import SeqIO
from Bio.Graphics import BasicChromosome

selected = df[df["genotype_id"]=="B73v4"]
entries = selected[["chr", "len"]].values.tolist()
max_len = selected["len"].max()
telomere_length = 0

chr_diagram = BasicChromosome.Organism()
chr_diagram.page_size = (29.7 * cm, 21 * cm)  # A4 landscape

for index, (name, length) in enumerate(entries):
    
    # For each chrosome generate a feature list of tuples (start, start, strand, name, color,)
    features = []
    for row in coords[coords["gene_chr"]==name].values:
        features.append((row[3], row[3], None, row[0], "black",))
    
    cur_chromosome = BasicChromosome.Chromosome(name)
    # Set the scale to the MAXIMUM length plus the two telomeres in bp,
    # want the same scale used on all five chromosomes so they can be
    # compared to each other
    cur_chromosome.scale_num = max_len + 2

    # Record an Artemis style integer color in the feature's qualifiers,
    # 1 = Black, 2 = Red, 3 = Green, 4 = blue, 5 =cyan, 6 = purple
    
    # The features can either be SeqFeature objects, or tuples of values: start (int), 
    # end (int), strand (+1, -1, O or None), label (string), ReportLab color (string or object), 
    # and optional ReportLab fill color.
    
    # This will it to every chromsome. Make chromsome specific lists
    
    
    # Add an opening telomere
    start = BasicChromosome.TelomereSegment()
    start.scale = telomere_length
    cur_chromosome.add(start)
    
    # Add a body - again using bp as the scale length here.
    body = BasicChromosome.AnnotatedChromosomeSegment(length, features)
    body.scale = length
    cur_chromosome.add(body)
    
    # Add a closing telomere
    end = BasicChromosome.TelomereSegment(inverted=True)
    end.scale = telomere_length
    cur_chromosome.add(end)
    
    # This chromosome is done
    chr_diagram.add(cur_chromosome)

    from reportlab.graphics import renderPM
reportlab = chr_diagram.draw(None, "Genome Diagram")
renderPM.drawToFile(reportlab, 'example1.png', 'PNG')
#chr_diagram.draw("test_chrom.pdf", "Genome Diagram")

In [55]:
chr_diagram.draw(None, "Genome Diagram")